In [1]:
import torch
import torch.nn as nn
import numpy as np
import sympy
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'../..')))

from Select import select
from Random_choice import random_choice
from All import all0
from Rmse_loss import rmse_loss
from Train_fun import train_fun

torch.set_default_dtype(torch.float32)
device = torch.device('cuda:0')
np.set_printoptions(suppress=True)
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
def diff_FN(netw,cor_train):
    i = 0
    cache = np.zeros((1,7))
    while i*10000<len(cor_train): 
        pts = cor_train[i*10000:(i+1)*10000].clone().detach().to(device)
        pts.requires_grad_(True)
        outs1 = netw(pts)
        grad = torch.autograd.grad(outs1, pts, grad_outputs=torch.ones_like(outs1), create_graph=True)[0]
        wt_t = grad[:,0].reshape(-1,1) 
        wx_t = grad[:,1].reshape(-1,1)
        wy_t = grad[:,2].reshape(-1,1)
        wx = wx_t.cpu().detach().numpy()
        wy = wy_t.cpu().detach().numpy()
        wt = wt_t.cpu().detach().numpy()
        w_pred = outs1.cpu().detach().numpy()
        del wx_t, wt_t, wy_t, grad, outs1
        torch.cuda.empty_cache()

        outs1 = netw(pts)
        grad = torch.autograd.grad(outs1, pts, grad_outputs=torch.ones_like(outs1), create_graph=True)[0]
        wxx_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs1), create_graph=True)[0])[:,1].reshape(-1,1)
        wxy_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs1), create_graph=True)[0])[:,2].reshape(-1,1)    
        wxx = wxx_t.cpu().detach().numpy()
        wxy = wxy_t.cpu().detach().numpy()
        del wxx_t, wxy_t

        wyy_t= (torch.autograd.grad(grad[:,2].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs1), create_graph=True)[0])[:,2].reshape(-1,1)
        wyy = wyy_t.cpu().detach().numpy()
        del wyy_t, grad, outs1
        torch.cuda.empty_cache()
        cache = np.vstack((cache,np.hstack((w_pred,wx,wy,wxx,wxy,wyy,wt))))
        i=i+1
    return cache[1:]

In [4]:
class FourNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64,64)
        self.linear4 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = self.linear4(x)
        return x
        
class ThreeNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 64)
        self.linear2 = nn.Linear(64,64)
        self.linear3 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = self.linear3(x)
        return x  

In [5]:
data_IC1 = scipy.io.loadmat('FN_IC2_Avoid104TS.mat')
Exact_u_IC1 = np.real(data_IC1['u'])
Exact_v_IC1 = np.real(data_IC1['v'])
t_steps = np.real(data_IC1['t'].flatten()[:,None])
x_steps = np.real(data_IC1['x'].flatten()[:,None])
y_steps = np.real(data_IC1['y'].flatten()[:,None])

In [6]:
xx, yy, tt = np.meshgrid(x_steps, y_steps, t_steps)
X_star = np.hstack((tt.flatten()[:,None], xx.flatten()[:,None], yy.flatten()[:,None]))
U_star_IC1 = np.hstack((Exact_u_IC1.flatten()[:,None], Exact_v_IC1.flatten()[:,None]))

In [8]:
data = np.hstack((X_star,U_star_IC1))
xx, yy, tt = np.meshgrid(x_steps[1:-1], y_steps[1:-1], t_steps[1:-1])
X_star = np.hstack((tt.flatten()[:,None], xx.flatten()[:,None], yy.flatten()[:,None]))
U_star_IC1 = np.hstack((Exact_u_IC1[1:-1,1:-1,1:-1].flatten()[:,None], 
                        Exact_v_IC1[1:-1,1:-1,1:-1].flatten()[:,None]))
data = np.hstack((X_star,U_star_IC1))

In [10]:
data = data[data[:,0]<21]
data = data[data[:,1]<50]
data = data[data[:,2]<50]
#data = data[np.round((data[:,0]-7.18)/0.03).astype('int')%6==0]
#data = data[np.round((data[:,1]-0.5)/0.5).astype('int')%2==0]
#data = data[np.round((data[:,2]-0.5)/0.5).astype('int')%2==0]
data = data.astype('float32')

In [11]:
per = 0.2
np.random.seed(0)
cor_train = torch.from_numpy(data[:,:3])
u_train = torch.from_numpy(data[:,3])
u_clean = data[:,3].reshape(-1,1)
u_noise = u_clean + per*np.std(u_clean)*np.random.randn(u_clean.shape[0],u_clean.shape[1])
u_noise = u_noise.astype('float32')
u_train = torch.from_numpy(u_noise)

v_clean = data[:,4].reshape(-1,1)
v_noise = v_clean + per*np.std(v_clean)*np.random.randn(v_clean.shape[0],v_clean.shape[1])
v_noise = v_noise.astype('float32')
v_train = torch.from_numpy(v_noise)

In [12]:
netu = FourNet_sin(3).to(device)
try:
    netu.load_state_dict(torch.load('net/netu_{}'.format(per)))
except:
    train_fun(netu,cor_train,u_train,N_red_lr=4, epochs=1000,lr=0.01)
    torch.save(netu.state_dict(),'net/netu_{}'.format(per))

In [13]:
netv = ThreeNet_sin(3).to(device)
try:
    netv.load_state_dict(torch.load('net/netv_{}'.format(per)))
except:
    train_fun(netv,cor_train,v_train,N_red_lr=4, epochs=1000,lr=0.01)
    torch.save(netv.state_dict(),'net/netv_{}'.format(per))

In [14]:
cache = diff_FN(netu,cor_train)
u_pred = cache[:,0].reshape(-1,1)
ut = cache[:,-1].reshape(-1,1)
ux = cache[:,1].reshape(-1,1)
uy = cache[:,2].reshape(-1,1)
uxx = cache[:,3].reshape(-1,1)
uxy = cache[:,4].reshape(-1,1)
uyy= cache[:,5].reshape(-1,1)

cache = diff_FN(netv,cor_train)
v_pred = cache[:,0].reshape(-1,1)
vt = cache[:,-1].reshape(-1,1)
vx = cache[:,1].reshape(-1,1)
vy = cache[:,2].reshape(-1,1)
vxx = cache[:,3].reshape(-1,1)
vxy = cache[:,4].reshape(-1,1)
vyy= cache[:,5].reshape(-1,1)

In [15]:
rmse_loss(uxx+uyy+u_pred-u_pred**3-v_clean+0.01,ut)

tensor(0.1265, device='cuda:0', dtype=torch.float64)

In [35]:
feature = np.hstack((u_pred,v_pred,ux,uy,uxx,uxy,uyy,vx,vy,vxx,vxy,vyy,ut))
feature = random_choice(feature, 100000)
LC,exp_list,exp = all0('resultu_{}.txt'.format(per),feature,
                              state=2,deg=3,tol=0.05,alpha=10**-5,
                              niterations=500,data_points=500)

ModuleNotFoundError: No module named 'pysr'

Started!

Expressions evaluated per second: 6.610e+05
Head worker occupation: 7.2%
Progress: 1524 / 7500 total iterations (20.320%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.359e-02  1.594e+01  y = -0.088965
3           2.491e-02  1.495e-01  y = (-0.33551 * x₃)
5           2.345e-02  3.015e-02  y = ((-0.33647 * x₃) - x₅)
7           1.354e-02  2.747e-01  y = (((0.41436 * x₀) - x₃) * 0.55069)
8           1.186e-03  2.435e+00  y = (x₀ - (x₃ + cube(x₀)))
10          1.951e-04  9.025e-01  y = (x₅ - (x₃ + (cube(x₀) - x₀)))
12          8.980e-05  3.881e-01  y = ((x₅ - (x₃ + (cube(x₀) - x₀))) + 0.010263)
14          8.854e-05  7.026e-03  y = ((-0.99395 * (x₃ + (cube(x₀) - (x₀ + x₅)))) + 0.0096635)
16          8.845e-05  5.113e-04  y = (((-0.99395 + x₇) * (x₃ + ((cube(x₀) - x₅) - x₀))) + 0.009...
                                  3101)
17          8.776e-05  7.873e-03

Started!

Expressions evaluated per second: 6.340e+05
Head worker occupation: 10.2%
Progress: 1500 / 7500 total iterations (20.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.659e-02  1.594e+01  y = -0.078765
3           2.890e-02  1.180e-01  y = (x₄ * -0.31243)
5           2.643e-02  4.471e-02  y = ((x₀ * 0.3892) - x₄)
7           1.576e-02  2.586e-01  y = (((x₀ / 2.3004) - x₄) / 1.5352)
8           1.563e-04  4.613e+00  y = ((x₀ - cube(x₀)) - x₄)
10          5.471e-05  5.249e-01  y = (0.010081 - (x₄ - (x₀ - cube(x₀))))
12          5.457e-05  1.330e-03  y = (0.010081 - (x₄ - (x₀ - cube(x₀ - x₈))))
14          5.379e-05  7.150e-03  y = ((0.038734 * x₃) - ((x₄ - (x₀ - cube(x₀))) + -0.010085))
16          5.356e-05  2.176e-03  y = (((x₉ * x₃) / -0.043035) - ((x₄ + -0.010105) - (x₀ - cube(...
                                  x₀))))
18          5.355e-05  1.447e-

Started!

Expressions evaluated per second: 6.240e+05
Head worker occupation: 9.4%
Progress: 1391 / 7500 total iterations (18.547%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.521e-02  1.594e+01  y = -0.084915
3           3.493e-02  4.037e-03  y = (x₄ + -0.085691)
5           3.304e-02  2.786e-02  y = ((x₄ * 14.856) - 0.095882)
7           1.453e-02  4.106e-01  y = ((-1.1542 + (x₃ * x₃)) * 0.34674)
9           1.414e-02  1.367e-02  y = ((-1.1552 + ((x₃ + x₁) * x₃)) * 0.34708)
10          1.358e-02  4.078e-02  y = ((-0.89911 + sin(x₃ * x₃)) * 0.50024)
11          1.079e-02  2.292e-01  y = ((-1.1568 + ((x₃ * x₃) - x₀)) * (0.36871 - x₀))
13          1.035e-02  2.122e-02  y = (((-1.1568 + (x₃ * x₃)) - (x₀ / 0.65009)) * (0.36796 - x₀)...
                                  )
15          7.647e-03  1.512e-01  y = (((-1.1568 + (x₃ * x₃)) - x₀) * ((0.39324 - (x₃ * 0.20642

In [41]:
k = select(LC,0.05)
exp_list[k]

[1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1]
[-1 -1 -1 -1 -1 -1]


-x0**3 + x0 - x4 + 0.010080835